# Double-ended queue

# Principe

Formellement, une "double-ended queue" ou **deque** est un type de donnée abstrait qui permet insertion et suppression de données à chaque bout (tête et queue).

Plusieurs mises en oeuvres sont possibles parmi celles vues précédemment

* liste doublement chainée
* tableau circulaire
* tableau circulaire dynamique

Ici, on s'intéresse plus particulièrement à l'approche mise en oeuvre par `std::deque<T>` en C++: un **tableau dynamique de tableaux** 

Les données sont stockées dans plusieurs petits tableaux de capacité fixe `chunk_cap` alloués dynamiquement: les **chunks**
    
Les addresses de ces chunks sont elles-même stockées dans un buffer circulaire, la **map**, de capacité variable `map_cap`. 

Le premier élément est stocké à l'indice `chunk_debut` du chunk dont l'addresse est stockée dans l'emplacement `map_debut` de la map. 

Les suivants sont stockés aux indices suivants de ce chunk, puis dans les chunks suivants à partir de l'indice 0. 

Enfin, le nombre total d'éléments de la deque est stocké dans `taille`.

Ces 5 attributs et l'addresse du début du tableau `map` alloué dynamiquement suffisent à localiser tout élément en mémoire. 

Ecrivons cette classe `DeQue`. Le constructeur prend en paramètre la capacité fixe des chunks et la capacité de départ de la map. 

In [28]:
class DeQue:
    def __init__(self,chunk_cap,map_cap):
        self.map = [None]*map_cap
        self.map_cap = map_cap 
        self.chunk_cap = chunk_cap
        self.map_debut = 0
        self.taille = 0
        self.chunk_debut = 0

## Indices physiques

Comme dans la mise en oeuvre du buffer circulaire, il est essentiel de pouvoir calculer les indices physiques à partir de l'indice logique `i` dans `[0,n-1]` pour `n` éléments.

Il y en a ici deux

* `i_chunk`, la position de `i` dans son chunk. 

* `i_map`, le position du chunk de `i` dans la map

On calcule ce dernier en passant par le résultat intermédiaire `i_logique_map`, l'indice logique du chunk dans le buffer circulaire `map`.

In [29]:
def indices_physiques(deq,i):
    
    i_chunk = ( i + deq.chunk_debut ) % deq.chunk_cap
  
    i_logique_map = ( i + deq.chunk_debut ) // deq.chunk_cap
    
    i_map = ( i_logique_map + deq.map_debut ) % deq.map_cap 
    
    return i_chunk, i_map 

## Insertion en queue

Ignorons pour le moment le problème de gestion de la capacité de `map`

In [30]:
def check_capacite(deq,taille_demandee): pass

pour insérer en queue, il faut 

* calculer les indices physiques
* allouer le chunk si nécessaire
* écrire à l'emplacement `.map[i_map][i_chunk]`

In [31]:
def inserer_en_queue(deq,val):
    check_capacite(deq,deq.taille+1)
    
    i_chunk, i_map = indices_physiques(deq,deq.taille)
    if deq.map[i_map] == None: 
        deq.map[i_map] = [None] * deq.chunk_cap
    deq.map[i_map][i_chunk] = val
    deq.taille += 1
    
DeQue.append = inserer_en_queue

Pour observer comment la `DeQue` se remplit, écrivons une fonction affichant son contenu physique.

In [32]:
def afficher_deque(deq):
    
    str = "cc:{} mc:{} cd:{} md:{} t:{} \n".format(
                         deq.chunk_cap, deq.map_cap, 
                         deq.chunk_debut, deq.map_debut, 
                         deq.taille)
    
    for i in range(deq.map_cap):
        if deq.map[i] == None:
            str += "None\n"
        else:
            str += deq.map[i].__str__() + "\n"
            
    return str
        
DeQue.__str__ = afficher_deque

In [46]:
D = DeQue(8,6)
for i in range(12):
    inserer_en_queue(D,i*2)
print(D)

cc:8 mc:6 cd:0 md:0 t:12 
[0, 2, 4, 6, 8, 10, 12, 14]
[16, 18, 20, 22, None, None, None, None]
None
None
None
None



## Insertion en tête

L'insertion en tête fonctionne de manière similaire. La principale différence est qu'il faut mettre à jour `map_debut` et `chunk_debut`

In [47]:
def inserer_en_tete(deq,val):
    check_capacite(deq,deq.taille+1)
    
    i_chunk, i_map = indices_physiques(deq,-1)
    if deq.map[i_map] == None: 
        deq.map[i_map] = [None] * deq.chunk_cap
    deq.map[i_map][i_chunk] = val
    deq.map_debut = i_map
    deq.chunk_debut = i_chunk
    deq.taille += 1
    
DeQue.append_left = inserer_en_tete

Observons son effet sur le contenu de la deque

In [48]:
for i in range(1,15):
    D.append_left(-i)
print(D)

cc:8 mc:6 cd:2 md:4 t:26 
[0, 2, 4, 6, 8, 10, 12, 14]
[16, 18, 20, 22, None, None, None, None]
None
None
[None, None, -14, -13, -12, -11, -10, -9]
[-8, -7, -6, -5, -4, -3, -2, -1]



## Accès indexé

L'accès indexé est immédiat à partir du calcul des indices physiques

In [49]:
def get_item(deq,i):
    if i < 0 or i >= deq.taille: raise IndexError()
        
    i_chunk, i_map = indices_physiques(deq,i)
    return deq.map[i_map][i_chunk]
DeQue.__getitem__ = get_item

In [50]:
def set_item(deq,i,val):
    if i < 0 or i >= deq.taille: raise IndexError()
        
    i_chunk, i_map = indices_physiques(deq,i)
    deq.map[i_map][i_chunk] = val
DeQue.__setitem__ = set_item

In [59]:
for i,v in enumerate(D):
    print("D[{}] = {}".format(i,v).ljust(13), end = "")
    if i % 4 == 3: print()

D[0] = -14   D[1] = -13   D[2] = -12   D[3] = -11   
D[4] = -10   D[5] = -9    D[6] = -8    D[7] = -7    
D[8] = -6    D[9] = -5    D[10] = -4   D[11] = -3   
D[12] = -2   D[13] = -1   D[14] = 0    D[15] = 2    
D[16] = 4    D[17] = 6    D[18] = 8    D[19] = 10   
D[20] = 12   D[21] = 14   D[22] = 16   D[23] = 18   
D[24] = 20   D[25] = 22   

In [10]:
for i,v in enumerate(D):
    D[i] = 20-v
for i,v in enumerate(D):
    print("D[{}] = {}".format(i,v))

D[0] = 20
D[1] = 18
D[2] = 16
D[3] = 14
D[4] = 12
D[5] = 10


In [37]:
def check_capacite(deq,taille_demandee):
    if taille_demandee <= (deq.map_cap-1) * deq.chunk_cap: 
        return 
    nouvelle_map_cap = deq.map_cap * 2 if deq.map_cap != 0 else 1
    while (nouvelle_map_cap-1) * deq.chunk_cap < taille_demandee:
        nouvelle_map_cap *= 2
    augmente_map_cap(deq,nouvelle_map_cap)    
        
def augmente_map_cap(deq,nouvelle_map_cap):
    nouvelle_map = [None] * nouvelle_map_cap
    nouveau_debut = nouvelle_map_cap-deq.map_cap+deq.map_debut
    
    nouvelle_map[0:deq.map_debut] = deq.map[0:deq.map_debut]
    nouvelle_map[nouveau_debut:nouvelle_map_cap] = deq.map[deq.map_debut:deq.map_cap]

    deq.map = nouvelle_map
    deq.map_cap = nouvelle_map_cap
    deq.map_debut = nouveau_debut

In [13]:
for i,v in enumerate(D):
    print("D[{}] = {}".format(i,v))

D[0] = -5
D[1] = -4
D[2] = -3
D[3] = -2
D[4] = -1
D[5] = 20
D[6] = 18
D[7] = 16
D[8] = 14
D[9] = 12
D[10] = 10


In [14]:
for i in range(1,6):
    D.append(i)
print(D)

cc:4 mc:8 cd:3 md:6 t:16 
[20, 18, 16, 14]
[12, 10, 1, 2]
[3, 4, 5, None]
None
None
None
[None, None, None, -5]
[-4, -3, -2, -1]



In [15]:
for i,v in enumerate(D):
    print("D[{}] = {}".format(i,v))

D[0] = -5
D[1] = -4
D[2] = -3
D[3] = -2
D[4] = -1
D[5] = 20
D[6] = 18
D[7] = 16
D[8] = 14
D[9] = 12
D[10] = 10
D[11] = 1
D[12] = 2
D[13] = 3
D[14] = 4
D[15] = 5


In [16]:
def supprimer_en_queue(deq):
    if deq.taille < 1: raise IndexError()
    i_chunk, i_map = indices_physiques(deq,deq.taille-1)
    deq.map[i_map][i_chunk] = None
    deq.taille -= 1
    
DeQue.pop = supprimer_en_queue

In [17]:
print(D)
for v in D: print(v,end=" ")
print("\n")
for i in range(3):
    D.pop()
print(D)
for v in D: print(v,end=" ")
print("\n")

cc:4 mc:8 cd:3 md:6 t:16 
[20, 18, 16, 14]
[12, 10, 1, 2]
[3, 4, 5, None]
None
None
None
[None, None, None, -5]
[-4, -3, -2, -1]

-5 -4 -3 -2 -1 20 18 16 14 12 10 1 2 3 4 5 

cc:4 mc:8 cd:3 md:6 t:13 
[20, 18, 16, 14]
[12, 10, 1, 2]
[None, None, None, None]
None
None
None
[None, None, None, -5]
[-4, -3, -2, -1]

-5 -4 -3 -2 -1 20 18 16 14 12 10 1 2 



In [18]:
def supprimer_en_tete(deq):
    if deq.taille < 1: raise IndexError()
    i_chunk, i_map = indices_physiques(deq,0)
    deq.map[i_map][i_chunk] = None
    i_chunk, i_map = indices_physiques(deq,1)
    deq.taille -= 1
    deq.map_debut = i_map
    deq.chunk_debut = i_chunk
    
DeQue.popleft = supprimer_en_tete

In [19]:
print(D)
for v in D: print(v,end=" ")
print("\n")
for i in range(3):
    D.popleft()
print(D)
for v in D: print(v,end=" ")
print("\n")

cc:4 mc:8 cd:3 md:6 t:13 
[20, 18, 16, 14]
[12, 10, 1, 2]
[None, None, None, None]
None
None
None
[None, None, None, -5]
[-4, -3, -2, -1]

-5 -4 -3 -2 -1 20 18 16 14 12 10 1 2 

cc:4 mc:8 cd:2 md:7 t:10 
[20, 18, 16, 14]
[12, 10, 1, 2]
[None, None, None, None]
None
None
None
[None, None, None, None]
[None, None, -2, -1]

-2 -1 20 18 16 14 12 10 1 2 



## Complexités 

temporelle et spatiale

## Conclusion

<table style="width: 100%; border: 0px">
<tr style="background-color:white; border:0px">
<td style="width: 120px; border: 0px">
    <img src="https://heig-vd.ch/ResourcePackages/WhiteFox/assets/images/logo-heig-vd.svg" height=200px align=left >
    </td>
    <td style="vertical-align: middle; border: 0px" height=200px>
    <p style="text-align: left">
        <a href="https://ocuisenaire.github.io/ASD1-notebooks/">ASD1 Notebooks on GitHub.io</a>
 </p>        
<p style="text-align: left">
© Olivier Cuisenaire, 2018 </p>
</td>
</tr>
</table>